In [1]:
#!pip install tqdm

In [1]:
from tqdm import tqdm
import melee
import os
import glob
import torch
import pickle
import json
import multiprocessing as mp
from util import make_obs, get_controller_state

In [3]:
def process_slp(slp, output_path):
    states = []
    actions = []
    game_id = os.path.basename(slp).split(".")[0]
    #console = melee.Console(system="file", allow_old_version=False, path=slp)
    console = melee.Console(system="file", path=slp)
    console.connect()
    while True:
        gs = console.step()
        if gs is None:
            break

        controller_state = gs.players[1].controller_state  # or whichever field holds the input
        act = get_controller_state(controller_state)

        obs = make_obs(gs, max_projectiles=5)
        states.append(obs)
        actions.append(act)

    # store the states and actions in a pkl file
    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, f"{game_id}.pkl"), "wb") as f:
        pickle.dump({"states": states, "actions": actions}, f)

    return len(actions)

In [4]:
# REPLACE WITH YOUR PATH NAME
source_path = "replays"
#source_path = "data"
# find all slp files in the data folder recursively
slp_paths = glob.glob(os.path.join(source_path, "**/*.slp"), recursive=True)
print(len(slp_paths))

# output_path = "data/processed"
train_output_path = "data/processed"
# val_output_path = "/media/summertony717/Samsung_T5/val_mini_50"

515


In [5]:
def make_train(x): return process_slp(x, train_output_path)
# def make_val(x): return process_slp(x, val_output_path)

# use multiprocessing to speed up the process
print(f'working with {mp.cpu_count()} cores')
with mp.Pool(processes=mp.cpu_count()) as pool:
    train = pool.map(make_train, slp_paths)
    # val = pool.map(make_val, slp_paths[300:350])

metadata = {}
for k, v in zip(slp_paths, train):
    metadata[os.path.basename(k)] = v

with open(os.path.join(train_output_path, "metadata.json"), "w") as f:
    json.dump(metadata, f)

# metadata = {}
# for k, v in zip(slp_paths[300:350], val):
#     metadata[os.path.basename(k)] = v

# with open(os.path.join(val_output_path, "metadata.json"), "w") as f:
#     json.dump(metadata, f)


working with 12 cores


/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
corrupted size vs. prev_size
/home/summertony717/School/cs224r/project/libmelee/melee/console.py:976: RuntimeWarning: overflow encountered in scalar add
  projectile.owner = np.ndarray((1,), ">B", event_bytes, 0x2A)[0] + 1
/

KeyboardInterrupt: 